In [1]:
import zmq
import mmap
import numpy as np

In [2]:
filename = "zmqdump.dat"

In [3]:
context = zmq.Context()
zmq_socket = context.socket(zmq.PUSH)
HOST = '127.0.0.1'
zmq_socket.bind(f"tcp://{HOST}:5014")

<SocketContext(bind='tcp://127.0.0.1:5014')>

In [10]:
with open(filename, mode='rb') as f:
    mm = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    try:
        index = 0
        while index < len(mm):
            len_field_bytes = mm[index:index+8]
            len_field = np.frombuffer(len_field_bytes, dtype=np.int64, count=1)
            index += 8
            data = mm[index:index+len_field[0]]
            zmq_socket.send(data)
            index += len(data)
    finally:
        mm.close()